In [1]:
from bokeh.plotting import output_notebook
output_notebook()

In [2]:
from dataclasses import dataclass
import numpy as np

@dataclass
class PolicySet:
    IMI: float = .1
    IRS: float = .23
    healthcare_budget: int = 20_000_000
    education_budget: int = 30_000_000

@dataclass
class Person:
    income: float
    
    def __init__(self): # Unrealistic model generating incomes used for testing purposes
        self.income = np.random.uniform(low=0, high=10_000)
        
    def tax_burden(self, policy_set: PolicySet) -> float: # total i.e. absolute tax_burden
        irs_total = policy_set.IRS * self.income
        imi_total = max(self.income-1_000,0) * policy_set.IMI
        return min(irs_total + imi_total, self.income) # unrealistic model used for testing purposes 

@dataclass
class Budget:
    total: int
    healthcare: int
    education: int
    
    @property
    def deficit(self):
        return self.total - sum([self.healthcare, self.education])

In [3]:
from tqdm import tqdm

def budget_calculator(
        policy_set: PolicySet,
        population_size: int = 10_500_000
) -> Budget:
    tax_contribution_list = [
        Person().tax_burden(policy_set)
        for _
        in tqdm(range(min(population_size,10**5)))
    ]
    revenue = sum(tax_contribution_list)
    return Budget(
        total=revenue,
        healthcare=PolicySet.healthcare_budget,
        education=PolicySet.education_budget,
    )

In [4]:
budget = budget_calculator(policy_set=PolicySet())
budget

In [5]:
from matplotlib import pyplot as plt
ps = PolicySet()
observations = [
    (p.income, np.divide(p.tax_burden(ps), p.income))
     for p
     in [Person() for _ in tqdm(range(10**5))] 
]
[income_array, tax_burden_array] = list(zip(*observations))

# Plot
fig, ax = plt.subplots(figsize = (15,2))
ax.scatter(income_array, tax_burden_array, s=1)
ax.set_xscale("log")

In [6]:
ps_1 = PolicySet()
ps_2 = PolicySet(IRS=.4)

budget_as_dict = {
    "total": budget.total,
    "healthcare": budget.healthcare,
    "education": budget.education,
    # "deficit": budget.deficit,
}
# plt.barh(budget_as_dict.keys(), budget_as_dict.values())
plt.bar(x=budget_as_dict.keys(), height=budget_as_dict.values())

In [7]:
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import figure, show

x = np.linspace(0, 10, 500)
y = np.sin(x)

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(y_range=(-10, 10), width=400, height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

amp = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude")
freq = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency")
phase = Slider(start=-6.4, end=6.4, value=0, step=.1, title="Phase")
offset = Slider(start=-9, end=9, value=0, step=.1, title="Offset")

callback = CustomJS(
    args=dict(source=source, amp=amp, freq=freq, phase=phase, offset=offset),
    code="""
        const A = amp.value
        const k = freq.value
        const phi = phase.value
        const B = offset.value
    
        const x = source.data.x
        const y = Array.from(x, (x) => B + A*Math.sin(k*x+phi))
        source.data = { x, y }
        """
)

amp.js_on_change('value', callback)
freq.js_on_change('value', callback)
phase.js_on_change('value', callback)
offset.js_on_change('value', callback)

show(
    row(
        plot,
        column(amp, freq, phase, offset)
    )
)

In [8]:
from bokeh.plotting import figure, show

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
counts = [5, 3, 4, 2, 4, 6]

p = figure(
    x_range=fruits,
    height=350,
    title="Fruit Counts",
    toolbar_location=None,
    tools=""
)

p.vbar(
    x=fruits,
    top=counts,
    width=0.9
)

p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)
